# Chapter 3: Replication and other controllers: deploying managed pods


## Kubectl Commands
* kubectl logs {name} -- previous: Shows the logs of the previous resource
* kubectl label {resource} {name} {label_key}={label_value} --overwrite: Overwrites an existing label
* kubectl edit rc {name}: Edits a rc's yaml config in the default text editor
* kubectl scale rc {name} --replicas={number}: Rescales a replica to the desired number of replicas
* kubectl delete rc {name}: Deletes a replication controller and its pods
* kubectl delete rc {name} --cascade=False: Deletes only the replication controller (not the pods)

## Examing Liveness Probes
* When a pod is scheduled to a node, the node's Kubelet will run the container for as long as the pod exists
    * If the container's main process crashes, then the Kubelet restarts the container
    * If there are bugs in a container, then k8s will restart it automatically (self-healing)

* Liveness Probes
    * A way for K8s to check if a container is still alive
    * Main Mechanisms:
        * GET on the conatiner's IP to see if it returns a 2xx or a 3xx response
        * TCP Socket connection to the specified container port to see if there's a response
        * EXEC probe executes an arbitrary command in the container and checks for an exit code
    * When a container is killed, a new container is created
        * Not the same container being restarted
    * It's pretty easy to setup parameter configurations for each probe
    * Liveness checks should check only the internals of the app
        * Ex: If a database goes down then a probe can keep on failing which won't fix the problem

* Exit Codes are program error indicators
    * 128 + x
        * SIGKILL is 9 which is code 137
        * SIGETERM is 15 which is code 143

```
# Liveness Probe Example
apiVersion: v1
kind: Pod
metadata:
  name: kubia-liveness
spec:
  containers:
  - image: luksa/kubia-unhealthy
    name: kubia
    livenessProbe:
      httpGet:
        path: /
        port: 8080
        initialDelaySeconds: 15
```

## Examining ReplicationControllers
* ReplicationController
    * Kubernetes resource that ensures that the desired state of pods is always kept running
        * If a pod goes missing/down, the RC will move to replace that pod
        * If there are too many pods, the RC will remove the excess replicas
    * RCs are meant to create/manage replices of a pod
    * Components:
        * Label Selector: Determines what pods are in the RC's scope
        * Replica Count: Specifices the desired number of pods that should be running
        * Pod Template: Used when creating new pod replicas
    * Changing the Components:
        * Changing the label selector makes the existing pods fall out of the RC scope
            * So the currently-managed pods now behave like manually created pods
        * Changing the replica count will alter the existing state of the pods
        * Changing the pod template only affects new pods created by the RC
            * You'll have to delete the old pods manually so that the RC can spin up new ones
    * Benefits of using an RC:
        * Ensures that a pod/replicas are always running
        * Ensures that cluster nodes are always running
        * Enables easy horizontal scaling of pods
    * Normally you can do *kubectl delete* to delete both the RC and its pods
        * If you want to keep the pods running, then you can set a cascade parameter

```
# Replication Controller Example
apiVersion: v1
kind: ReplicationController
metadata:
  name: kubia
spec:
  replicas: 3
  selector:
    app: kubia
template:
  metadata:
    labels:
      app: kubia
  spec:
    containers:
    - name: kubia
      image: luksa/kubia
      ports:
        - containerPort: 8080
```

## Examining ReplicaSets
* Seen as the successor to ReplicationControllers
    * RCs will be eventually deprecated
* RSs are are almost exactly the same as RCs
* ReplicaSets vs. ReplicationController
    * RS allow for more expressive pod selectors
        * RC label selectors only work on pods that include a certain label
        * RS label selectors allow for matching based on presence/lack of a label or of the value
        * RS label selectors can also group matching expressions together
* Expression Types:
  * matchLabels
    * Similar to RC configs where it must match a given key:value
  * matchExpressions
    * In: Label's value must match one of the specified values
    * Not IN: Label's value must not match any of the specified values
    * Exists: Pods must include a label with a specified key
    * DoesNotExist: Pods must not include a label with a specified key
    * If there are multiple expressions, all of them must evaluate to true to match a pod

```
# ReplicaSet Example
apiVersion: apps/v1beta2
kind: ReplicaSet
metadata:
  name: kubia
spec:
  replicas: 3
  selector:
    matchExpressions:
      - key: app
        operator: In
        values:
          - kubia
template:
    metadata:
        labels:
        app: kubia
    spec:
        containers:
        - name: kubia
            image: luksa/kubia
```


## Examining DaemonSets
* Sometimes you will want to apply a pod on every cluster node
  * Ex: Infrastructure-related pods like log collectors and resource monitors
* DaemonSets are similar to RCs and RSs but they aren't scattered around the cluster randomly
* DaemonSets create as many pods as there are nodes
* DaemonSets create pods from the pod template configured in it
* DaemonSets can also be deployed to a subset of all nodes

```
# DaemonSet Example
apiVersion: apps/v1beta2
kind: DaemonSet
metadata:
  name: ssd-monitor
spec:
  selector:
    matchLabels:
      app: ssd-monitor
  template:
    metadata:
      labels:
        app: ssd-monitor
    spec:
      nodeSelector:
        disk: ssd
      containers:
      - name: main
        image: luksa/ssd-monitor
```

## Examining Job Resources
* With RCs, RSs, and DCs, these all run continuously and are never considered completed.
  * When they exit from their process, the pods are restarted
* Job Resources:
  * Allows for the running of pods (whose container isn't restarted) when the process running inside finishes successfully. 
  * If there are node failures, the pods on the node managed by a job can be rescheduled like with RSs
  * Pods managed by jobs are rescheduled until they finish successfully!
* Important Job Resources Parameters
  * *completions*: How many times you want the Job's pod to run sequentially
  * *parallelism*: How many pods you want to run in parallel
  * *activeDeadlineSeconds*: Max amount of time for a job to take
* Job Resources can also manage CronJobs
  * Cron jobs: Jobs that need to be executed at a specific time / time interval

```
# Example Job Resource
apiVersion: batch/v1
kind: Job
metadata:
  name: batch-job-every-fifteen-minutes
spec:
  completions: 5
  parallelism: 2
  schedule: "0,15,30,45 * * * *"
  jobTemplate:
    spec:
      template:
        metadata:
          labels:
            app: periodic-batch-job
        spec:
          restartPolicy: OnFailure
          containers:
          - name: main
            image: luksa/batch-job

  ```

## Summary
* You can specify a liveness probe to have Kubernetes restart your container as soon as it’s no longer healthy (where the app defines what’s considered healthy).
* Pods shouldn’t be created directly, because they will not be re-created if they’re deleted by mistake, if the node they’re running on fails, or if they’re evicted from the node.
* ReplicationControllers always keep the desired number of pod replicas running.
* Scaling pods horizontally is as easy as changing the desired replica count on a ReplicationController.
* Pods aren’t owned by the ReplicationControllers and can be moved between them if necessary.
* A ReplicationController creates new pods from a pod template. Changing the template has no effect on existing pods.
* ReplicationControllers should be replaced with ReplicaSets and Deployments, which provide the same functionality, but with additional powerful features.
* ReplicationControllers and ReplicaSets schedule pods to random cluster nodes, whereas DaemonSets make sure every node runs a single instance of a pod defined in the DaemonSet.
* Pods that perform a batch task should be created through a Kubernetes Job resource, not directly or through a ReplicationController or similar object.
* Jobs that need to run sometime in the future can be created through CronJob resources.